In [1]:
import pandas as pd
import numpy as np
import time
from time import mktime
from datetime import datetime, timedelta
import geopy.distance
import gc
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

import data_prep_functions
import request_api_nba_stats

### BASES PARA PREDICT

In [2]:
all_games = pd.read_csv("all_games_nba_18_19.csv")

In [3]:
all_games

,GAME,GAME_ID_home,TEAM_ID_home,TEAM_NAME_home,team_home,TEAM_CITY_home,FGM_home,FGA_home,FG_PCT_home,FG3M_home,...,DATE,SEASON,fl_playoff,fl_home_win,team_home_game_num,team_away_game_num,DISTANCE_KM_home,DISTANCE_KM_away,DAYS_FROM_LAST_GAME_home,DAYS_FROM_LAST_GAME_away
0,PHI @ BOS 2018-10-16,21800001,1.610613e+09,Celtics,BOS,Boston,42,97,0.433,11,...,2018-10-16,2019,0,1,1,1,0.000000,440.371911,NaN,NaN
1,OKC @ GSW 2018-10-16,21800002,1.610613e+09,Warriors,GSW,Golden State,42,95,0.442,7,...,2018-10-16,2019,0,1,1,1,0.000000,2216.822719,NaN,NaN
2,MIL @ CHA 2018-10-17,21800003,1.610613e+09,Hornets,CHA,Charlotte,41,92,0.446,16,...,2018-10-17,2019,0,0,1,1,0.000000,1061.082957,NaN,NaN
3,BKN @ DET 2018-10-17,21800004,1.610613e+09,Pistons,DET,Detroit,39,92,0.424,6,...,2018-10-17,2019,0,1,1,1,0.000000,803.100363,NaN,NaN
4,MEM @ IND 2018-10-17,21800005,1.610613e+09,Pacers,IND,Indiana,47,83,0.566,10,...,2018-10-17,2019,0,1,1,1,0.000000,618.174461,NaN,NaN
5,MIA @ ORL 2018-10-17,21800006,1.610613e+09,Magic,ORL,Orlando,39,94,0.415,7,...,2018-10-17,2019,0,1,1,1,0.000000,327.752956,NaN,NaN
6,ATL @ NYK 2018-10-17,21800007,1.610613e+09,Knicks,NYK,New York,45,99,0.455,12,...,2018-10-17,2019,0,1,1,1,0.000000,1204.295148,NaN,NaN
7,CLE @ TOR 2018-10-17,21800008,1.610613e+09,Raptors,TOR,Toronto,45,92,0.489,14,...,2018-10-17,2019,0,1,1,1,0.000000,304.607627,NaN,NaN
8,NOP @ HOU 2018-10-17,21800009,1.610613e+09,Rockets,HOU,Houston,39,92,0.424,16,...,2018-10-17,2019,0,0,1,1,0.000000,510.639860,NaN,NaN
9,MIN @ SAS 2018-10-17,21800010,1.610613e+09,Spurs,SAS,San Antonio,40,93,0.430,11,...,2018-10-17,2019,0,1,1,1,0.000000,1784.844575,NaN,NaN


#### Fluxo Atualiza Rodada de Ontem

In [ ]:
# AMANHÃ = datetime.fromtimestamp(mktime(time.localtime(time.time() + 24*3600)))
# HOJE = datetime.today()

In [172]:
games_ids = request_api_nba_stats.get_list_gameids(datetime.today() - timedelta(1), start_at=len(all_games)) 

In [173]:
df_full, df_full_jogo, erros = request_api_nba_stats.get_nba_stats_data(games_ids)

In [174]:
new_base_nba_2018 = pd.concat(df_full_jogo).reset_index()

In [176]:
new_all_games = data_prep_functions.prepara_base(new_base_nba_2018)
new_all_games = data_prep_functions.cria_features(new_all_games)

In [178]:
#all_games = new_all_games
all_games = pd.concat([all_games, new_all_games])

In [145]:
# resp = data_prep_functions.gera_last_N_games(all_games, N=[5])
# delta_L5_2018 = data_prep_functions.variaveis_delta(resp)
# delta_L5_2018.to_csv("delta_cross_L5_nba_18_19.csv")

In [ ]:
all_games["DATE"] = [datetime.strptime(x, '%Y-%m-%d') for x in all_games["DATE"]]

In [17]:
all_games.to_csv("all_games_nba_18_19.csv")

### Fluxo Predição no Dia

In [4]:
new_games_ids = request_api_nba_stats.get_list_gameids(max_date = datetime.fromtimestamp(mktime(time.localtime(time.time() + 24*3600)))
                                                   , year = '2018', start_at=len(all_games))

In [5]:
new_games = request_api_nba_stats.get_game_infos_dates(new_games_ids)

In [16]:
new_games = data_prep_functions.cria_features(new_games, all_games)

In [18]:
to_predict = data_prep_functions.gera_last_N_games(new_games, all_games)

In [19]:
to_predict = data_prep_functions.variaveis_delta(to_predict)

In [20]:
to_predict[['D2_PLUS_MINUS_L5', 'D1_WIN_PCT_L5']]

,D2_PLUS_MINUS_L5,D1_WIN_PCT_L5
ATL @ CHA 2018-11-06,24.4,0.4
WAS @ DAL 2018-11-06,11.6,-0.2
BKN @ PHX 2018-11-06,-1.2,-0.2
MIL @ POR 2018-11-06,4.8,0.0
OKC @ CLE 2018-11-07,-10.0,-0.8
DET @ ORL 2018-11-07,-13.0,0.4
NYK @ ATL 2018-11-07,1.2,-0.2
SAS @ MIA 2018-11-07,0.6,-0.4
PHI @ IND 2018-11-07,22.2,0.0
DEN @ MEM 2018-11-07,5.0,-0.4
